Основная структура сети взята отсюда: https://github.com/Lasagne/Recipes/blob/master/papers/deep_residual_learning/Deep_Residual_Learning_CIFAR-10.py

In [1]:
import sys
import os
import time
import string
import random
import pickle

import numpy as np
import theano
import theano.tensor as T
import lasagne
import matplotlib.pyplot as pl

%matplotlib inline

Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GRID K520 (0000:00:03.0)


Выгружаем данные и делаем аугментацию: вычитаем среднее и добавляем поворты.

In [2]:
from cifar import load_CIFAR10

dir_path = './cifar10/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(dir_path)

mean = np.mean(X_train[0:50000], axis=0)
X_train -= mean
X_test -= mean

X_train_flip = X_train[:, :, :, ::-1]
y_train_flip = y_train
X_train = np.concatenate((X_train, X_train_flip), axis=0)
y_train = np.concatenate((y_train, y_train_flip), axis=0)

In [3]:
input_X = T.tensor4("X")
target_y = T.vector("target Y integer", dtype='int32')

Определяем сеть

В отличае от версии откуда был взят основной костяк по-другому уменьшаю размерность, когда делаю residual connection. В той версии из квадрата 2х2 брался верхний левый пиксель, а недостающая половина фильтров заполнялась нулями. Я в первую половину фильтров кладу верхний левый пиксель из квадрата 2х2, а во вторую половину нижний правый.

In [14]:
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.layers import ElemwiseSumLayer
from lasagne.layers import InputLayer
from lasagne.layers import DenseLayer
from lasagne.layers import GlobalPoolLayer
from lasagne.layers import PadLayer
from lasagne.layers import ExpressionLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import MaxPool2DLayer as MaxPool
from lasagne.nonlinearities import softmax, rectify
from lasagne.layers import batch_norm

def build_cnn(input_var=None, n=5):
    # create a residual learning building block with two stacked 3x3 convlayers
    def residual_block(net, increase_dim=False):
        input_num_filters = net.output_shape[1]
        if increase_dim:
            first_stride = (2,2)
            out_num_filters = input_num_filters * 2
        else:
            first_stride = (1,1)
            out_num_filters = input_num_filters

        old_net = net
        net = batch_norm(ConvLayer(net, num_filters=out_num_filters,
                                       filter_size=3, stride=first_stride,
                                       pad='same',
                                       W=lasagne.init.HeNormal(gain='relu'),
                                       flip_filters=False))

        net = batch_norm(ConvLayer(net, num_filters=out_num_filters,
                                       filter_size=3,
                                       nonlinearity=None, pad='same',
                                       W=lasagne.init.HeNormal(gain='relu'),
                                       flip_filters=False))
        
        if increase_dim:
            old_net = ExpressionLayer(old_net, lambda X: T.concatenate([X[:, :, ::2, ::2],
                                                                         X[:, :, 1::2, 1::2]],
                                                                        axis=1),
                                      lambda s: (s[0], 2*s[1], int(s[2]/2), int(s[3]/2)))
            net = NonlinearityLayer(ElemwiseSumLayer([net, old_net]))
        else:
            net = NonlinearityLayer(ElemwiseSumLayer([net, old_net]))
        
        return net

    net = InputLayer(shape=(None, 3, 32, 32), input_var=input_var)

    net = batch_norm(ConvLayer(net, num_filters=16, filter_size=3, pad='same',
                             W=lasagne.init.HeNormal(gain='relu'), flip_filters=False))
    
    for _ in range(n):
        net = residual_block(net)

    net = residual_block(net, increase_dim=True)
    for _ in range(1, n):
        net = residual_block(net)

    net = residual_block(net, increase_dim=True)
    for _ in range(1, n):
        net = residual_block(net)
    
    net = GlobalPoolLayer(net)

    network = DenseLayer(net, num_units=10, W=lasagne.init.Normal(),
                         nonlinearity=softmax)

    return network

In [15]:
def iterate_minibatches(inputs, targets, batch_size, shuffle=False, train=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batch_size]
        else:
            excerpt = slice(start_idx, start_idx + batch_size)
        if train:
            padded = np.pad(inputs[excerpt], ((0, 0), (0, 0), (4, 4), (4, 4)), mode='edge')
            random_cropped = np.zeros(inputs[excerpt].shape, dtype=np.float32)
            crops = np.random.randint(0, 9, size=(batch_size, 2))
            for r in range(batch_size):
                random_cropped[r, :, :, :] = padded[r, :, crops[r, 0]:(crops[r, 0] + 32),
                                                    crops[r, 1]:(crops[r, 1] + 32)]
            inp_exc = random_cropped
        else:
            inp_exc = inputs[excerpt]

        yield inp_exc, targets[excerpt]

In [16]:
net = build_cnn(input_X)

In [17]:
y_predicted = lasagne.layers.get_output(net)
all_weights = lasagne.layers.get_all_params(net, trainable=True)
all_layers = lasagne.layers.get_all_layers(net)
print(all_weights)

[W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, b]


In [18]:
reg_lambda = 0.0001
current_learning_rate = 0.1
loss = lasagne.objectives.categorical_crossentropy(y_predicted, target_y).mean()
loss += reg_lambda * lasagne.regularization.regularize_layer_params(all_layers, lasagne.regularization.l2)
accuracy = lasagne.objectives.categorical_accuracy(y_predicted, target_y).mean()

In [19]:
grad_update = lasagne.updates.momentum(loss, all_weights, learning_rate=current_learning_rate, momentum=0.9)

In [20]:
train_fun = theano.function([input_X, target_y], [loss, accuracy], updates=grad_update, allow_input_downcast=True)

In [21]:
accuracy_fun = theano.function([input_X, target_y], accuracy, allow_input_downcast=True)

Обучаемся, при этом делаем следующий learning rate scheduling: уменьшаем learning rate на 40, 60 эпохе в 10 раз, на 82 эпохе меняем updates на адама, и на 94 еще уменьшаем learinig rate в 10 раз.

In [22]:
import time

epohs_number = 100
batch_size = 128 #размер мини-батча

for epoch in range(epohs_number):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batch_size, shuffle=True, train=True):
        inputs, targets = batch
        train_err_batch, train_acc_batch = train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, y_test, 512):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(epoch + 1, epohs_number, time.time() - start_time))
    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))
        
    if epoch == 40 or epoch == 60:
        current_learning_rate /= 10
        grad_update = lasagne.updates.momentum(loss, all_weights, 
                                               learning_rate=current_learning_rate)
        train_fun = theano.function([input_X, target_y], [loss, accuracy],
                                    updates=grad_update, allow_input_downcast=True)

    if epoch == 82:
        grad_update = lasagne.updates.adam(loss, all_weights)
        train_fun = theano.function([input_X, target_y], [loss, accuracy],
                                    updates=grad_update, allow_input_downcast=True)
    if epoch == 94:
        current_learning_rate /= 10
        grad_update = lasagne.updates.adam(loss, all_weights,
                                           learning_rate=current_learning_rate)
        train_fun = theano.function([input_X, target_y], [loss, accuracy],
                                    updates=grad_update, allow_input_downcast=True)

Epoch 1 of 100 took 429.312s
  training loss (in-iteration):		2.279851
  train accuracy:		27.01 %
  validation accuracy:		49.54 %
Epoch 2 of 100 took 428.680s
  training loss (in-iteration):		1.383674
  train accuracy:		60.00 %
  validation accuracy:		70.13 %
Epoch 3 of 100 took 429.442s
  training loss (in-iteration):		1.052085
  train accuracy:		72.17 %
  validation accuracy:		76.74 %
Epoch 4 of 100 took 428.646s
  training loss (in-iteration):		0.903806
  train accuracy:		77.02 %
  validation accuracy:		80.06 %
Epoch 5 of 100 took 429.417s
  training loss (in-iteration):		0.818005
  train accuracy:		79.96 %
  validation accuracy:		80.50 %
Epoch 6 of 100 took 428.629s
  training loss (in-iteration):		0.768917
  train accuracy:		81.56 %
  validation accuracy:		82.75 %
Epoch 7 of 100 took 429.370s
  training loss (in-iteration):		0.729460
  train accuracy:		82.91 %
  validation accuracy:		83.93 %
Epoch 8 of 100 took 428.593s
  training loss (in-iteration):		0.701009
  train accuracy:		

Продолжаем учиться. 92.52 уже было на 98 эпохе, но хочется еще сохранить параметры сети.

In [24]:
import time

epohs_number = 100
batch_size = 128 #размер мини-батча

for epoch in range(epohs_number):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batch_size, shuffle=True, train=True):
        inputs, targets = batch
        train_err_batch, train_acc_batch = train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, y_test, 512):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(epoch + 101, epohs_number, time.time() - start_time))
    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))

Epoch 101 of 100 took 212.040s
  training loss (in-iteration):		0.225001
  train accuracy:		99.07 %
  validation accuracy:		92.49 %


KeyboardInterrupt: 

В предыдущей ячейке забыл сделать break при условии достижения порога. Здесь номер эпохи сдвинут на 1 (101 -> 102)

In [25]:
import time

epohs_number = 100
batch_size = 128 #размер мини-батча

for epoch in range(epohs_number):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batch_size, shuffle=True, train=True):
        inputs, targets = batch
        train_err_batch, train_acc_batch = train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, y_test, 512):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(epoch + 101, epohs_number, time.time() - start_time))
    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))
    
    if (val_acc / val_batches * 100) > 92.5:
        break

Epoch 101 of 100 took 212.071s
  training loss (in-iteration):		0.218150
  train accuracy:		99.15 %
  validation accuracy:		92.60 %


Удостоверяюсь в том, что я действительно колдун 80 уровня

In [26]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 512):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 92.5:
    print("Achievement unlocked: колдун 80 уровня")
else:
    print("Нужно больше магии!")

Final results:
  test accuracy:		92.60 %
Achievement unlocked: колдун 80 уровня


И сохраняю параметры сети

In [27]:
np.savez('model.npz', *lasagne.layers.get_all_param_values(net))

Продолжаю учиться, так как интересно какого результата можно достичь (тут на самом деле номера эпох сдвинуты на 2 (101 -> 103)

In [28]:
import time

epohs_number = 100
batch_size = 128 #размер мини-батча

for epoch in range(epohs_number):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batch_size, shuffle=True, train=True):
        inputs, targets = batch
        train_err_batch, train_acc_batch = train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, y_test, 512):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(epoch + 101, epohs_number, time.time() - start_time))
    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))

    if epoch == 8:
        current_learning_rate /= 10
        grad_update = lasagne.updates.adam(loss, all_weights,
                                           learning_rate=current_learning_rate)
        train_fun = theano.function([input_X, target_y], [loss, accuracy],
                                    updates=grad_update, allow_input_downcast=True)

Epoch 101 of 100 took 212.104s
  training loss (in-iteration):		0.212733
  train accuracy:		99.21 %
  validation accuracy:		92.40 %
Epoch 102 of 100 took 212.048s
  training loss (in-iteration):		0.207756
  train accuracy:		99.28 %
  validation accuracy:		92.51 %
Epoch 103 of 100 took 212.016s
  training loss (in-iteration):		0.203082
  train accuracy:		99.29 %
  validation accuracy:		92.58 %
Epoch 104 of 100 took 211.973s
  training loss (in-iteration):		0.198290
  train accuracy:		99.35 %
  validation accuracy:		92.37 %
Epoch 105 of 100 took 212.119s
  training loss (in-iteration):		0.194381
  train accuracy:		99.41 %
  validation accuracy:		92.60 %
Epoch 106 of 100 took 212.082s
  training loss (in-iteration):		0.190053
  train accuracy:		99.43 %
  validation accuracy:		92.48 %
Epoch 107 of 100 took 212.000s
  training loss (in-iteration):		0.186839
  train accuracy:		99.42 %
  validation accuracy:		92.36 %
Epoch 108 of 100 took 212.006s
  training loss (in-iteration):		0.182741
  t

KeyboardInterrupt: 

Максимум был 92.85 на 118 эпохе